# Imports

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
import torch
import os
from collections import OrderedDict

In [ ]:
DEVICE = torch.device('cuda')

# Load models and rename layers

In [ ]:
from tqdm.auto import tqdm

In [ ]:
%run ../models/checkpoint/__init__.py
%run ../utils/files.py

## Rename function

In [ ]:
def rename_layer(name):
    name = name.replace('attention_layer', 'attention')
    name = name.replace('lstm_cell', 'word_lstm')
    name = name.replace('W_vocab', 'word_fc')
    name = name.replace('embeddings_table', 'word_embeddings')
    name = name.replace('features_fc.2', 'features_fc')
    
    name = name.replace('stop_control.0', 'stop_control')
    name = name.replace('last_fc.1', 'last_fc')

    return name

In [ ]:
def test_rename_layers(layers):
    new_layers = []
    for layer in layers:
        layer2 = rename_layer(layer)
        if layer2 not in layers:
            new_layers.append((layer, layer2))
        if layer2 != layer:
            print(f'{layer:<42} {layer2 if layer2 != layer else "SAME"}')
    return new_layers

## Rename

In [ ]:
run_ids = [
    # RunId('0430_231758', True, 'rg'),
    # RunId('0428_200424', True, 'rg'),
    RunId(name, False, 'rg')
    for name in os.listdir(_get_parent_folder('models', False, 'rg'))
    if name != 'debug'
]
len(run_ids)

In [ ]:
for run_id in tqdm(run_ids):
    folder = get_checkpoint_folder(run_id, save_mode=False, assert_exists=True)

    for filepath in os.listdir(folder):
        if not 'checkpoint' in filepath:
            continue
        filepath = os.path.join(folder, filepath)
        
        # Load checkpoint
        checkpoint = torch.load(filepath, map_location=DEVICE)
        
        # Rename layers
        state_dict = checkpoint['model']
        new_state_dict = OrderedDict()
        for key, value in state_dict.items():
            new_key = rename_layer(key)
            new_state_dict[new_key] = value
        
        # Override layers and save
        checkpoint['model'] = new_state_dict
        torch.save(checkpoint, filepath2) # TODO: change filepath2 by a new one